In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from glob import glob
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, UpSampling2D, UpSampling1D, Concatenate, Cropping1D, TimeDistributed, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from more_itertools import chunked
from sklearn.metrics import r2_score
from keras.regularizers import l2
from keras.layers import Bidirectional
import gc
import polars as pl

features = ['state_t_0', 'state_t_1', 'state_t_2', 'state_t_3', 'state_t_4', 'state_t_5', 'state_t_6', 'state_t_7', 'state_t_8', 'state_t_9', 'state_t_10', 'state_t_11', 'state_t_12', 'state_t_13', 'state_t_14', 'state_t_15', 'state_t_16', 'state_t_17', 'state_t_18', 'state_t_19', 'state_t_20', 'state_t_21', 'state_t_22', 'state_t_23', 'state_t_24', 'state_t_25', 'state_t_26', 'state_t_27', 'state_t_28', 'state_t_29', 'state_t_30', 'state_t_31', 'state_t_32', 'state_t_33', 'state_t_34', 'state_t_35', 'state_t_36', 'state_t_37', 'state_t_38', 'state_t_39', 'state_t_40', 'state_t_41', 'state_t_42', 'state_t_43', 'state_t_44', 'state_t_45', 'state_t_46', 'state_t_47', 'state_t_48', 'state_t_49', 'state_t_50', 'state_t_51', 'state_t_52', 'state_t_53', 'state_t_54', 'state_t_55', 'state_t_56', 'state_t_57', 'state_t_58', 'state_t_59', 'state_q0001_0', 'state_q0001_1', 'state_q0001_2', 'state_q0001_3', 'state_q0001_4', 'state_q0001_5', 'state_q0001_6', 'state_q0001_7', 'state_q0001_8', 'state_q0001_9', 'state_q0001_10', 'state_q0001_11', 'state_q0001_12', 'state_q0001_13', 'state_q0001_14', 'state_q0001_15', 'state_q0001_16', 'state_q0001_17', 'state_q0001_18', 'state_q0001_19', 'state_q0001_20', 'state_q0001_21', 'state_q0001_22', 'state_q0001_23', 'state_q0001_24', 'state_q0001_25', 'state_q0001_26', 'state_q0001_27', 'state_q0001_28', 'state_q0001_29', 'state_q0001_30', 'state_q0001_31', 'state_q0001_32', 'state_q0001_33', 'state_q0001_34', 'state_q0001_35', 'state_q0001_36', 'state_q0001_37', 'state_q0001_38', 'state_q0001_39', 'state_q0001_40', 'state_q0001_41', 'state_q0001_42', 'state_q0001_43', 'state_q0001_44', 'state_q0001_45', 'state_q0001_46', 'state_q0001_47', 'state_q0001_48', 'state_q0001_49', 'state_q0001_50', 'state_q0001_51', 'state_q0001_52', 'state_q0001_53', 'state_q0001_54', 'state_q0001_55', 'state_q0001_56', 'state_q0001_57', 'state_q0001_58', 'state_q0001_59', 'state_q0002_0', 'state_q0002_1', 'state_q0002_2', 'state_q0002_3', 'state_q0002_4', 'state_q0002_5', 'state_q0002_6', 'state_q0002_7', 'state_q0002_8', 'state_q0002_9', 'state_q0002_10', 'state_q0002_11', 'state_q0002_12', 'state_q0002_13', 'state_q0002_14', 'state_q0002_15', 'state_q0002_16', 'state_q0002_17', 'state_q0002_18', 'state_q0002_19', 'state_q0002_20', 'state_q0002_21', 'state_q0002_22', 'state_q0002_23', 'state_q0002_24', 'state_q0002_25', 'state_q0002_26', 'state_q0002_27', 'state_q0002_28', 'state_q0002_29', 'state_q0002_30', 'state_q0002_31', 'state_q0002_32', 'state_q0002_33', 'state_q0002_34', 'state_q0002_35', 'state_q0002_36', 'state_q0002_37', 'state_q0002_38', 'state_q0002_39', 'state_q0002_40', 'state_q0002_41', 'state_q0002_42', 'state_q0002_43', 'state_q0002_44', 'state_q0002_45', 'state_q0002_46', 'state_q0002_47', 'state_q0002_48', 'state_q0002_49', 'state_q0002_50', 'state_q0002_51', 'state_q0002_52', 'state_q0002_53', 'state_q0002_54', 'state_q0002_55', 'state_q0002_56', 'state_q0002_57', 'state_q0002_58', 'state_q0002_59', 'state_q0003_0', 'state_q0003_1', 'state_q0003_2', 'state_q0003_3', 'state_q0003_4', 'state_q0003_5', 'state_q0003_6', 'state_q0003_7', 'state_q0003_8', 'state_q0003_9', 'state_q0003_10', 'state_q0003_11', 'state_q0003_12', 'state_q0003_13', 'state_q0003_14', 'state_q0003_15', 'state_q0003_16', 'state_q0003_17', 'state_q0003_18', 'state_q0003_19', 'state_q0003_20', 'state_q0003_21', 'state_q0003_22', 'state_q0003_23', 'state_q0003_24', 'state_q0003_25', 'state_q0003_26', 'state_q0003_27', 'state_q0003_28', 'state_q0003_29', 'state_q0003_30', 'state_q0003_31', 'state_q0003_32', 'state_q0003_33', 'state_q0003_34', 'state_q0003_35', 'state_q0003_36', 'state_q0003_37', 'state_q0003_38', 'state_q0003_39', 'state_q0003_40', 'state_q0003_41', 'state_q0003_42', 'state_q0003_43', 'state_q0003_44', 'state_q0003_45', 'state_q0003_46', 'state_q0003_47', 'state_q0003_48', 'state_q0003_49', 'state_q0003_50', 'state_q0003_51', 'state_q0003_52', 'state_q0003_53', 'state_q0003_54', 'state_q0003_55', 'state_q0003_56', 'state_q0003_57', 'state_q0003_58', 'state_q0003_59', 'state_u_0', 'state_u_1', 'state_u_2', 'state_u_3', 'state_u_4', 'state_u_5', 'state_u_6', 'state_u_7', 'state_u_8', 'state_u_9', 'state_u_10', 'state_u_11', 'state_u_12', 'state_u_13', 'state_u_14', 'state_u_15', 'state_u_16', 'state_u_17', 'state_u_18', 'state_u_19', 'state_u_20', 'state_u_21', 'state_u_22', 'state_u_23', 'state_u_24', 'state_u_25', 'state_u_26', 'state_u_27', 'state_u_28', 'state_u_29', 'state_u_30', 'state_u_31', 'state_u_32', 'state_u_33', 'state_u_34', 'state_u_35', 'state_u_36', 'state_u_37', 'state_u_38', 'state_u_39', 'state_u_40', 'state_u_41', 'state_u_42', 'state_u_43', 'state_u_44', 'state_u_45', 'state_u_46', 'state_u_47', 'state_u_48', 'state_u_49', 'state_u_50', 'state_u_51', 'state_u_52', 'state_u_53', 'state_u_54', 'state_u_55', 'state_u_56', 'state_u_57', 'state_u_58', 'state_u_59', 'state_v_0', 'state_v_1', 'state_v_2', 'state_v_3', 'state_v_4', 'state_v_5', 'state_v_6', 'state_v_7', 'state_v_8', 'state_v_9', 'state_v_10', 'state_v_11', 'state_v_12', 'state_v_13', 'state_v_14', 'state_v_15', 'state_v_16', 'state_v_17', 'state_v_18', 'state_v_19', 'state_v_20', 'state_v_21', 'state_v_22', 'state_v_23', 'state_v_24', 'state_v_25', 'state_v_26', 'state_v_27', 'state_v_28', 'state_v_29', 'state_v_30', 'state_v_31', 'state_v_32', 'state_v_33', 'state_v_34', 'state_v_35', 'state_v_36', 'state_v_37', 'state_v_38', 'state_v_39', 'state_v_40', 'state_v_41', 'state_v_42', 'state_v_43', 'state_v_44', 'state_v_45', 'state_v_46', 'state_v_47', 'state_v_48', 'state_v_49', 'state_v_50', 'state_v_51', 'state_v_52', 'state_v_53', 'state_v_54', 'state_v_55', 'state_v_56', 'state_v_57', 'state_v_58', 'state_v_59', 'state_ps', 'pbuf_SOLIN', 'pbuf_LHFLX', 'pbuf_SHFLX', 'pbuf_TAUX', 'pbuf_TAUY', 'pbuf_COSZRS', 'cam_in_ALDIF', 'cam_in_ALDIR', 'cam_in_ASDIF', 'cam_in_ASDIR', 'cam_in_LWUP', 'cam_in_ICEFRAC', 'cam_in_LANDFRAC', 'cam_in_OCNFRAC', 'cam_in_SNOWHLAND', 'pbuf_ozone_0', 'pbuf_ozone_1', 'pbuf_ozone_2', 'pbuf_ozone_3', 'pbuf_ozone_4', 'pbuf_ozone_5', 'pbuf_ozone_6', 'pbuf_ozone_7', 'pbuf_ozone_8', 'pbuf_ozone_9', 'pbuf_ozone_10', 'pbuf_ozone_11', 'pbuf_ozone_12', 'pbuf_ozone_13', 'pbuf_ozone_14', 'pbuf_ozone_15', 'pbuf_ozone_16', 'pbuf_ozone_17', 'pbuf_ozone_18', 'pbuf_ozone_19', 'pbuf_ozone_20', 'pbuf_ozone_21', 'pbuf_ozone_22', 'pbuf_ozone_23', 'pbuf_ozone_24', 'pbuf_ozone_25', 'pbuf_ozone_26', 'pbuf_ozone_27', 'pbuf_ozone_28', 'pbuf_ozone_29', 'pbuf_ozone_30', 'pbuf_ozone_31', 'pbuf_ozone_32', 'pbuf_ozone_33', 'pbuf_ozone_34', 'pbuf_ozone_35', 'pbuf_ozone_36', 'pbuf_ozone_37', 'pbuf_ozone_38', 'pbuf_ozone_39', 'pbuf_ozone_40', 'pbuf_ozone_41', 'pbuf_ozone_42', 'pbuf_ozone_43', 'pbuf_ozone_44', 'pbuf_ozone_45', 'pbuf_ozone_46', 'pbuf_ozone_47', 'pbuf_ozone_48', 'pbuf_ozone_49', 'pbuf_ozone_50', 'pbuf_ozone_51', 'pbuf_ozone_52', 'pbuf_ozone_53', 'pbuf_ozone_54', 'pbuf_ozone_55', 'pbuf_ozone_56', 'pbuf_ozone_57', 'pbuf_ozone_58', 'pbuf_ozone_59', 'pbuf_CH4_0', 'pbuf_CH4_1', 'pbuf_CH4_2', 'pbuf_CH4_3', 'pbuf_CH4_4', 'pbuf_CH4_5', 'pbuf_CH4_6', 'pbuf_CH4_7', 'pbuf_CH4_8', 'pbuf_CH4_9', 'pbuf_CH4_10', 'pbuf_CH4_11', 'pbuf_CH4_12', 'pbuf_CH4_13', 'pbuf_CH4_14', 'pbuf_CH4_15', 'pbuf_CH4_16', 'pbuf_CH4_17', 'pbuf_CH4_18', 'pbuf_CH4_19', 'pbuf_CH4_20', 'pbuf_CH4_21', 'pbuf_CH4_22', 'pbuf_CH4_23', 'pbuf_CH4_24', 'pbuf_CH4_25', 'pbuf_CH4_26', 'pbuf_N2O_0', 'pbuf_N2O_1', 'pbuf_N2O_2', 'pbuf_N2O_3', 'pbuf_N2O_4', 'pbuf_N2O_5', 'pbuf_N2O_6', 'pbuf_N2O_7', 'pbuf_N2O_8', 'pbuf_N2O_9', 'pbuf_N2O_10', 'pbuf_N2O_11', 'pbuf_N2O_12', 'pbuf_N2O_13', 'pbuf_N2O_14', 'pbuf_N2O_15', 'pbuf_N2O_16', 'pbuf_N2O_17', 'pbuf_N2O_18', 'pbuf_N2O_19', 'pbuf_N2O_20', 'pbuf_N2O_21', 'pbuf_N2O_22', 'pbuf_N2O_23', 'pbuf_N2O_24', 'pbuf_N2O_25', 'pbuf_N2O_26']
unpredictable_targets = ['ptend_q0002_15', 'ptend_q0002_16', 'ptend_q0002_17', 'ptend_q0002_18', 'ptend_q0002_19', 'ptend_q0002_20', 'ptend_q0002_21', 'ptend_q0002_22']
targets = ['ptend_t_0', 'ptend_t_1', 'ptend_t_2', 'ptend_t_3', 'ptend_t_4', 'ptend_t_5', 'ptend_t_6', 'ptend_t_7', 'ptend_t_8', 'ptend_t_9', 'ptend_t_10', 'ptend_t_11', 'ptend_t_12', 'ptend_t_13', 'ptend_t_14', 'ptend_t_15', 'ptend_t_16', 'ptend_t_17', 'ptend_t_18', 'ptend_t_19', 'ptend_t_20', 'ptend_t_21', 'ptend_t_22', 'ptend_t_23', 'ptend_t_24', 'ptend_t_25', 'ptend_t_26', 'ptend_t_27', 'ptend_t_28', 'ptend_t_29', 'ptend_t_30', 'ptend_t_31', 'ptend_t_32', 'ptend_t_33', 'ptend_t_34', 'ptend_t_35', 'ptend_t_36', 'ptend_t_37', 'ptend_t_38', 'ptend_t_39', 'ptend_t_40', 'ptend_t_41', 'ptend_t_42', 'ptend_t_43', 'ptend_t_44', 'ptend_t_45', 'ptend_t_46', 'ptend_t_47', 'ptend_t_48', 'ptend_t_49', 'ptend_t_50', 'ptend_t_51', 'ptend_t_52', 'ptend_t_53', 'ptend_t_54', 'ptend_t_55', 'ptend_t_56', 'ptend_t_57', 'ptend_t_58', 'ptend_t_59', 'ptend_q0001_0', 'ptend_q0001_1', 'ptend_q0001_2', 'ptend_q0001_3', 'ptend_q0001_4', 'ptend_q0001_5', 'ptend_q0001_6', 'ptend_q0001_7', 'ptend_q0001_8', 'ptend_q0001_9', 'ptend_q0001_10', 'ptend_q0001_11', 'ptend_q0001_12', 'ptend_q0001_13', 'ptend_q0001_14', 'ptend_q0001_15', 'ptend_q0001_16', 'ptend_q0001_17', 'ptend_q0001_18', 'ptend_q0001_19', 'ptend_q0001_20', 'ptend_q0001_21', 'ptend_q0001_22', 'ptend_q0001_23', 'ptend_q0001_24', 'ptend_q0001_25', 'ptend_q0001_26', 'ptend_q0001_27', 'ptend_q0001_28', 'ptend_q0001_29', 'ptend_q0001_30', 'ptend_q0001_31', 'ptend_q0001_32', 'ptend_q0001_33', 'ptend_q0001_34', 'ptend_q0001_35', 'ptend_q0001_36', 'ptend_q0001_37', 'ptend_q0001_38', 'ptend_q0001_39', 'ptend_q0001_40', 'ptend_q0001_41', 'ptend_q0001_42', 'ptend_q0001_43', 'ptend_q0001_44', 'ptend_q0001_45', 'ptend_q0001_46', 'ptend_q0001_47', 'ptend_q0001_48', 'ptend_q0001_49', 'ptend_q0001_50', 'ptend_q0001_51', 'ptend_q0001_52', 'ptend_q0001_53', 'ptend_q0001_54', 'ptend_q0001_55', 'ptend_q0001_56', 'ptend_q0001_57', 'ptend_q0001_58', 'ptend_q0001_59', 'ptend_q0002_0', 'ptend_q0002_1', 'ptend_q0002_2', 'ptend_q0002_3', 'ptend_q0002_4', 'ptend_q0002_5', 'ptend_q0002_6', 'ptend_q0002_7', 'ptend_q0002_8', 'ptend_q0002_9', 'ptend_q0002_10', 'ptend_q0002_11', 'ptend_q0002_12', 'ptend_q0002_13', 'ptend_q0002_14', 'ptend_q0002_15', 'ptend_q0002_16', 'ptend_q0002_17', 'ptend_q0002_18', 'ptend_q0002_19', 'ptend_q0002_20', 'ptend_q0002_21', 'ptend_q0002_22', 'ptend_q0002_23', 'ptend_q0002_24', 'ptend_q0002_25', 'ptend_q0002_26', 'ptend_q0002_27', 'ptend_q0002_28', 'ptend_q0002_29', 'ptend_q0002_30', 'ptend_q0002_31', 'ptend_q0002_32', 'ptend_q0002_33', 'ptend_q0002_34', 'ptend_q0002_35', 'ptend_q0002_36', 'ptend_q0002_37', 'ptend_q0002_38', 'ptend_q0002_39', 'ptend_q0002_40', 'ptend_q0002_41', 'ptend_q0002_42', 'ptend_q0002_43', 'ptend_q0002_44', 'ptend_q0002_45', 'ptend_q0002_46', 'ptend_q0002_47', 'ptend_q0002_48', 'ptend_q0002_49', 'ptend_q0002_50', 'ptend_q0002_51', 'ptend_q0002_52', 'ptend_q0002_53', 'ptend_q0002_54', 'ptend_q0002_55', 'ptend_q0002_56', 'ptend_q0002_57', 'ptend_q0002_58', 'ptend_q0002_59', 'ptend_q0003_0', 'ptend_q0003_1', 'ptend_q0003_2', 'ptend_q0003_3', 'ptend_q0003_4', 'ptend_q0003_5', 'ptend_q0003_6', 'ptend_q0003_7', 'ptend_q0003_8', 'ptend_q0003_9', 'ptend_q0003_10', 'ptend_q0003_11', 'ptend_q0003_12', 'ptend_q0003_13', 'ptend_q0003_14', 'ptend_q0003_15', 'ptend_q0003_16', 'ptend_q0003_17', 'ptend_q0003_18', 'ptend_q0003_19', 'ptend_q0003_20', 'ptend_q0003_21', 'ptend_q0003_22', 'ptend_q0003_23', 'ptend_q0003_24', 'ptend_q0003_25', 'ptend_q0003_26', 'ptend_q0003_27', 'ptend_q0003_28', 'ptend_q0003_29', 'ptend_q0003_30', 'ptend_q0003_31', 'ptend_q0003_32', 'ptend_q0003_33', 'ptend_q0003_34', 'ptend_q0003_35', 'ptend_q0003_36', 'ptend_q0003_37', 'ptend_q0003_38', 'ptend_q0003_39', 'ptend_q0003_40', 'ptend_q0003_41', 'ptend_q0003_42', 'ptend_q0003_43', 'ptend_q0003_44', 'ptend_q0003_45', 'ptend_q0003_46', 'ptend_q0003_47', 'ptend_q0003_48', 'ptend_q0003_49', 'ptend_q0003_50', 'ptend_q0003_51', 'ptend_q0003_52', 'ptend_q0003_53', 'ptend_q0003_54', 'ptend_q0003_55', 'ptend_q0003_56', 'ptend_q0003_57', 'ptend_q0003_58', 'ptend_q0003_59', 'ptend_u_0', 'ptend_u_1', 'ptend_u_2', 'ptend_u_3', 'ptend_u_4', 'ptend_u_5', 'ptend_u_6', 'ptend_u_7', 'ptend_u_8', 'ptend_u_9', 'ptend_u_10', 'ptend_u_11', 'ptend_u_12', 'ptend_u_13', 'ptend_u_14', 'ptend_u_15', 'ptend_u_16', 'ptend_u_17', 'ptend_u_18', 'ptend_u_19', 'ptend_u_20', 'ptend_u_21', 'ptend_u_22', 'ptend_u_23', 'ptend_u_24', 'ptend_u_25', 'ptend_u_26', 'ptend_u_27', 'ptend_u_28', 'ptend_u_29', 'ptend_u_30', 'ptend_u_31', 'ptend_u_32', 'ptend_u_33', 'ptend_u_34', 'ptend_u_35', 'ptend_u_36', 'ptend_u_37', 'ptend_u_38', 'ptend_u_39', 'ptend_u_40', 'ptend_u_41', 'ptend_u_42', 'ptend_u_43', 'ptend_u_44', 'ptend_u_45', 'ptend_u_46', 'ptend_u_47', 'ptend_u_48', 'ptend_u_49', 'ptend_u_50', 'ptend_u_51', 'ptend_u_52', 'ptend_u_53', 'ptend_u_54', 'ptend_u_55', 'ptend_u_56', 'ptend_u_57', 'ptend_u_58', 'ptend_u_59', 'ptend_v_0', 'ptend_v_1', 'ptend_v_2', 'ptend_v_3', 'ptend_v_4', 'ptend_v_5', 'ptend_v_6', 'ptend_v_7', 'ptend_v_8', 'ptend_v_9', 'ptend_v_10', 'ptend_v_11', 'ptend_v_12', 'ptend_v_13', 'ptend_v_14', 'ptend_v_15', 'ptend_v_16', 'ptend_v_17', 'ptend_v_18', 'ptend_v_19', 'ptend_v_20', 'ptend_v_21', 'ptend_v_22', 'ptend_v_23', 'ptend_v_24', 'ptend_v_25', 'ptend_v_26', 'ptend_v_27', 'ptend_v_28', 'ptend_v_29', 'ptend_v_30', 'ptend_v_31', 'ptend_v_32', 'ptend_v_33', 'ptend_v_34', 'ptend_v_35', 'ptend_v_36', 'ptend_v_37', 'ptend_v_38', 'ptend_v_39', 'ptend_v_40', 'ptend_v_41', 'ptend_v_42', 'ptend_v_43', 'ptend_v_44', 'ptend_v_45', 'ptend_v_46', 'ptend_v_47', 'ptend_v_48', 'ptend_v_49', 'ptend_v_50', 'ptend_v_51', 'ptend_v_52', 'ptend_v_53', 'ptend_v_54', 'ptend_v_55', 'ptend_v_56', 'ptend_v_57', 'ptend_v_58', 'ptend_v_59', 'cam_out_NETSW', 'cam_out_FLWDS', 'cam_out_PRECSC', 'cam_out_PRECC', 'cam_out_SOLS', 'cam_out_SOLL', 'cam_out_SOLSD', 'cam_out_SOLLD']
zero_w_targets = ['ptend_q0001_0', 'ptend_q0001_1', 'ptend_q0001_2', 'ptend_q0001_3', 'ptend_q0001_4', 'ptend_q0001_5', 'ptend_q0001_6', 'ptend_q0001_7', 'ptend_q0001_8', 'ptend_q0001_9', 'ptend_q0001_10', 'ptend_q0001_11', 'ptend_q0002_0', 'ptend_q0002_1', 'ptend_q0002_2', 'ptend_q0002_3', 'ptend_q0002_4', 'ptend_q0002_5', 'ptend_q0002_6', 'ptend_q0002_7', 'ptend_q0002_8', 'ptend_q0002_9', 'ptend_q0002_10', 'ptend_q0002_11', 'ptend_q0002_12', 'ptend_q0002_13', 'ptend_q0002_14', 'ptend_q0003_0', 'ptend_q0003_1', 'ptend_q0003_2', 'ptend_q0003_3', 'ptend_q0003_4', 'ptend_q0003_5', 'ptend_q0003_6', 'ptend_q0003_7', 'ptend_q0003_8', 'ptend_q0003_9', 'ptend_q0003_10', 'ptend_q0003_11', 'ptend_u_0', 'ptend_u_1', 'ptend_u_2', 'ptend_u_3', 'ptend_u_4', 'ptend_u_5', 'ptend_u_6', 'ptend_u_7', 'ptend_u_8', 'ptend_u_9', 'ptend_u_10', 'ptend_u_11', 'ptend_v_0', 'ptend_v_1', 'ptend_v_2', 'ptend_v_3', 'ptend_v_4', 'ptend_v_5', 'ptend_v_6', 'ptend_v_7', 'ptend_v_8', 'ptend_v_9', 'ptend_v_10', 'ptend_v_11']
# pred_targets = [tar for tar in targets if tar not in unpredictable_targets and tar not in zero_w_targets]
# pred_targets = [tar for tar in targets if tar not in zero_w_targets]

# train_files = sorted(glob("/kaggle/input/leap-train-parquet1/*.parquet"))
# train = pd.read_parquet(train_files[:1])
sample_df = pl.read_csv('/kaggle/input/leap-atmospheric-physics-ai-climsim/train.csv', n_rows=1)
column_names = sample_df.columns
column_names = column_names[1:]
del sample_df
# Create a dictionary where each column is mapped to pl.Float32
dtypes = {col: pl.Float32 for col in column_names}

train = pl.read_csv("/kaggle/input/leap-atmospheric-physics-ai-climsim/train.csv", n_rows=1200000, dtypes = dtypes)
# test = pd.read_parquet("/kaggle/input/leap-dataset-giba/test_batch/0.parquet")

# valid = pd.read_parquet("/kaggle/input/leap-train-parquet1/train_3.parquet")
# valid = pd.read_csv("/kaggle/input/leap-atmospheric-physics-ai-climsim/train.csv", skiprows = [600000], nrows = 600000)

weights = pd.read_csv("/kaggle/input/leap-atmospheric-physics-ai-climsim/sample_submission.csv", nrows=1)
del weights['sample_id']
weights = weights.T
weights = weights.to_dict()[0]

2024-06-12 02:54:05.159765: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 02:54:05.159878: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 02:54:05.289762: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
gc.collect()

0

In [3]:
# targets_ptend_t = [('ptend_t_' + str(i)) for i in range(60)]
# targets_ptend_q0001 = [('ptend_q0001_' + str(i)) for i in range(12,60)]
# targets_ptend_q0002 = [('ptend_q0002_') + str(i) for i in range(15,60)]
# targets_ptend_q0003 = [('ptend_q0003_') + str(i) for i in range(12,60)]
# targets_ptend_u = [('ptend_u_') + str(i) for i in range(12,60)]
# targets_ptend_v = [('ptend_v_') + str(i) for i in range(12,60)]
# single_targets = ['cam_out_NETSW', 'cam_out_FLWDS', 'cam_out_PRECSC', 'cam_out_PRECC', 'cam_out_SOLS', 'cam_out_SOLL', 'cam_out_SOLSD', 'cam_out_SOLLD']
# pred_targets = [targets_ptend_t, targets_ptend_q0001, targets_ptend_q0002, targets_ptend_q0003, targets_ptend_u, targets_ptend_v, single_targets]

In [4]:
# def reshape_data(timesteps, data):
#     remaining_rows =  data.shape[0] % timesteps
#     if remaining_rows > 0:
#         remaining_data = data[((data.shape[0] // timesteps) * timesteps):]
#         data =  data[:(data.shape[0] // timesteps)*timesteps].reshape(data.shape[0]//timesteps, timesteps, data.shape[1])
#         # Create padding with zeros
#         padding_needed = timesteps - remaining_rows
#         padding = np.zeros((padding_needed, data.shape[2]))
#         # Pad the remaining data
#         padded_remaining_data = np.vstack((remaining_data, padding))
#         # Reshape to match the required dimensions (1, timesteps, y)
#         padded_remaining_data = padded_remaining_data.reshape(1, timesteps, data.shape[2])
#         # Append the padded remaining data to the reshaped data
#         data = np.vstack((data, padded_remaining_data))
#         del remaining_data, padding_needed, padding, padded_remaining_data
#     else :
#         data = data[:(data.shape[0] // timesteps)*timesteps].reshape(data.shape[0] // timesteps, timesteps, data.shape[1])
#     gc.collect()
#     return data

In [5]:
# gc.collect()

In [6]:
# del X_train, X_valid

In [7]:
# train.columns[1:557]

In [8]:
# train.shape

In [9]:
# timesteps = 60
# scaler_y = StandardScaler()
# scaler_x = StandardScaler()

# # scaling of train and test features
# # X_train = scaler_x.fit_transform(train[features])
# X_train = scaler_x.fit_transform(train[train.columns[1:557]])
# # X_test = scaler_x.fit_transform(test[features])
# # X_valid = scaler_x.fit_transform(valid[features])

# # Reshaping train and test for LSTM model
# # X_train = reshape_data(timesteps, X_train)
# # # X_test = reshape_data(timesteps, X_test)
# # X_valid = reshape_data(timesteps, X_valid)
# gc.collect()

In [10]:
# y_train = scaler_y.fit_transform(train[targets])
# y_valid = scaler_y.fit_transform(valid[targets])
# y_train = reshape_data(timesteps, y_train)
# y_valid = reshape_data(timesteps, y_valid)

# Scaling 

In [11]:
gc.collect()

0

In [12]:
X_train = tf.convert_to_tensor(train[train.columns[1:557]], dtype=tf.float32)
y_train = tf.convert_to_tensor(train[targets], dtype=tf.float32)
normalizer_X = tf.keras.layers.Normalization(axis=-1)
normalizer_y = tf.keras.layers.Normalization(axis=-1)

# # Adapt the normalization layer to the features
normalizer_X.adapt(X_train)
normalizer_y.adapt(y_train)

mean_X = normalizer_X.mean
variance_X = normalizer_X.variance
stddev_X = normalizer_X.variance ** 0.5

mean_y = normalizer_y.mean
variance_y = normalizer_y.variance
stddev_y = tf.keras.ops.maximum(1e-10, normalizer_y.variance ** 0.5)

# Apply the normalization to the features
X_train = normalizer_X(X_train)
# y_train = normalizer_y(y_train)

In [13]:
y_train = (y_train - mean_y) / stddev_y

In [14]:
X_train.shape

TensorShape([1200000, 556])

In [15]:
y_train.shape

TensorShape([1200000, 368])

# Seq2seq Model Definition

In [16]:
from tensorflow.keras import layers
from tensorflow.keras import models
def build_cnn(activation='relu'):    
    return tf.keras.Sequential([
        tf.keras.layers.Conv1D(256, 3, padding='same', activation=activation),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv1D(128, 3, padding='same', activation=activation),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv1D(64, 3, padding='same', activation=activation),
        tf.keras.layers.BatchNormalization(),
    ])

# Define the input
input_tensor = tf.keras.Input(shape=(556,))

# input_tensor = tf.keras.Normalization(mean = mean_X, variance = variance_X)(input_tensor)
# Extract the first 360 rows, reshape to (6, 60), and transpose to (60, 6)
x1 = layers.Reshape((6, 60))(input_tensor[:, :360])
x1 = layers.Permute((2, 1))(x1)  # Transpose to (60, 6)
# Extract the next 120 rows, reshape to (2, 60), and transpose to (60, 2)
x2 = layers.Reshape((3, 60))(input_tensor[:, 376:556])
x2 = layers.Permute((2, 1))(x2)  # Transpose to (60, 2)
# Extract the remaining rows, reshape to (1, 10), and transpose to (60, 10)
x3 = layers.Reshape((1, 16))(input_tensor[:, 360:376])
x3 = layers.Lambda(lambda x: tf.tile(x, [1, 60, 1]))(x3)  # Duplicate the last 10 values 60 times to match (60, 10)
# Concatenate the reshaped tensors along the last axis to get (60, 18)
concatenated = layers.Concatenate(axis=-1)([x1, x2, x3])

# conv64 = layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(concatenated)
# conv3 = build_cnn(conv64)
# pool1 = layers.GlobalAveragePooling1D(keepdims=True)(conv3)
# pool2 = conv64 + conv3 + pool1
# pool2 = layers.BatchNormalization()(pool2)
# pool2 = pool2 + build_cnn(pool2);

# conv14 = layers.Conv1D(filters=14, kernel_size=1, padding='same')(pool2)
# p_seq = conv14[:, :, :6]
# p_seq = tf.keras.ops.transpose(p_seq, (0, 2, 1))
# p_seq = tf.keras.layers.Flatten()(p_seq)

# p_flat = conv14[:, :, 6:6 + 8]
# p_flat = tf.keras.ops.mean(p_flat, axis=1)
e = e0 = tf.keras.layers.Conv1D(64, 1, padding='same')(concatenated)
e = build_cnn()(e)
# add global average to allow some comunication between all levels even in a small CNN
e = e0 + e + tf.keras.layers.GlobalAveragePooling1D(keepdims=True)(e)
e = tf.keras.layers.BatchNormalization()(e)
e = e + build_cnn()(e)


p_all = tf.keras.layers.Conv1D(14, 1, padding='same')(e)

p_seq = p_all[:, :, :6]
p_seq = tf.keras.ops.transpose(p_seq, (0, 2, 1))
p_seq = tf.keras.layers.Flatten()(p_seq)
assert p_seq.shape[-1] == 360

p_flat = p_all[:, :, 6:6 + 8]
p_flat = tf.keras.ops.mean(p_flat, axis=1)
assert p_flat.shape[-1] == 8

P = tf.keras.ops.concatenate([p_seq, p_flat], axis=1)

# Define the model
model = models.Model(inputs=input_tensor, outputs=P)

# Print the model summary
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 556)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 360)       │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 180)       │          0 │ input_layer[0][0] │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_2          │ (None, 16)        │          0 │ input_layer[0][0] │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 6, 60)     │          0 │ get_item[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 3, 60)     │          0 │ get_item_1[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_2 (Reshape) │ (None, 1, 16)     │          0 │ get_item_2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ permute (Permute)   │ (None, 60, 6)     │          0 │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ permute_1 (Permute) │ (None, 60, 3)     │          0 │ reshape_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 60, 16)    │          0 │ reshape_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 60, 25)    │          0 │ permute[0][0],    │
│ (Concatenate)       │                   │            │ permute_1[0][0],  │
│                     │                   │            │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 60, 64)    │      1,664 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 60, 64)    │    174,272 │ conv1d[0][0]      │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 60, 64)    │          0 │ conv1d[0][0],     │
│                     │                   │            │ sequential[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1, 64)     │          0 │ sequential[0][0]  │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 60, 64)    │          0 │ add[0][0],        │
│                     │                   │            │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 64)    │        256 │ add_1[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 60, 64)    │    174,272 │ batch_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 60, 64)    │          0 │ batch_normalizat… │
│                     │                   │            │ sequential_1[0][

 Total params: 351,374 (1.34 MB)

 Trainable params: 349,454 (1.33 MB)

 Non-trainable params: 1,920 (7.50 KB)

In [17]:
# epochs = 20
# learning_rate = 1e-3
# BATCH_SIZE = 4096
# epochs_warmup = 1
# epochs_ending = 2
# steps_per_epoch = int(10 * 100_000 / BATCH_SIZE)

# lr_scheduler = tf.keras.optimizers.schedules.CosineDecay(
#     1e-4, 
#     (epochs - epochs_warmup - epochs_ending) * steps_per_epoch, 
#     warmup_target=learning_rate,
#     warmup_steps=steps_per_epoch * epochs_warmup,
#     alpha=0.1
# )

# plt.plot([lr_scheduler(it) for it in range(0, epochs * steps_per_epoch, steps_per_epoch)]);

# TRAINING

In [18]:
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=15, batch_size=4096, validation_split=0.2)

Epoch 1/15
  1/235 ━━━━━━━━━━━━━━━━━━━━ 1:47:24 28s/step - loss: 4.8180

I0000 00:00:1718160982.958324      73 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


235/235 ━━━━━━━━━━━━━━━━━━━━ 85s 247ms/step - loss: 0.9211 - val_loss: 0.5526
Epoch 2/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 44s 189ms/step - loss: 0.5875 - val_loss: 0.4776
Epoch 3/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 44s 189ms/step - loss: 0.5342 - val_loss: 0.4384
Epoch 4/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 44s 189ms/step - loss: 0.4939 - val_loss: 0.4181
Epoch 5/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 44s 189ms/step - loss: 0.4756 - val_loss: 0.3978
Epoch 6/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 44s 189ms/step - loss: 0.4583 - val_loss: 0.3808
Epoch 7/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 44s 189ms/step - loss: 0.4432 - val_loss: 0.3760
Epoch 8/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 44s 189ms/step - loss: 0.4379 - val_loss: 0.3707
Epoch 9/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 44s 189ms/step - loss: 0.4280 - val_loss: 0.3621
Epoch 10/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 44s 189ms/step - loss: 0.4220 - val_loss: 0.3596
Epoch 11/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 44s 189ms/step - loss: 0.4251 - val_loss: 0.3550
Epoch 12/15
235/235 ━━━━━━━━━━

In [19]:
gc.collect()

752

# Cross Validation

In [20]:
# valid = pd.read_csv("/kaggle/input/leap-atmospheric-physics-ai-climsim/train.csv", nrows = 100000, skiprows = 1200000, header = 0)
# X_valid = tf.convert_to_tensor(valid[valid.columns[1:557]], dtype=tf.float32)
# y_valid = tf.convert_to_tensor(valid[valid.columns[557:]], dtype=tf.float32)

# X_valid = normalizer_X(X_valid)
# y_valid = (y_valid - mean_y) / stddev_y

# predictions = model.predict(X_valid)

In [21]:
# def r2_score_cust(y_true, y_pred):
#     # Calculate the mean of the true values
#     y_mean = tf.reduce_mean(y_true)
#     # Calculate the total sum of squares (SS_tot)
#     ss_tot = tf.reduce_sum(tf.square(y_true - y_mean))
#     # Calculate the residual sum of squares (SS_res)
#     ss_res = tf.reduce_sum(tf.square(y_true - y_pred))
#     # Calculate R2 score
#     r2 = 1 - (ss_res / ss_tot)
#     return r2

# # Calculate R2 score
# r2 = r2_score_cust(y_valid, predictions)
# print("R2 Score:", r2)

In [22]:
# predictions_tf = tf.convert_to_tensor(predictions)
# predic_scaled = predictions * stddev_y.numpy() + mean_y.numpy()

In [23]:
# valid[valid.columns[557:]].head()

In [24]:
# # predictions_df = predictions
# # predictions_df = predictions_df * stddev + mean
# r2 = r2_score(valid[valid.columns[557:]], predic_scaled)
# print("R2 score after scaling : ",r2)

In [25]:
del X_train, y_train

In [26]:
test = pl.read_csv('/kaggle/input/leap-atmospheric-physics-ai-climsim/test.csv')

X_test = normalizer_X(tf.convert_to_tensor(test[test.columns[1:557]], dtype=tf.float32))
submission = model.predict(X_test)

submission = pd.DataFrame((submission * stddev_y + mean_y), columns=list(weights.keys()))
submission['sample_id'] = test['sample_id']
submission = submission[['sample_id'] + list(weights.keys())]

for idx in range(12, 30):
    submission[f"ptend_q0002_{idx}"] = -test[f"state_q0002_{idx}"].to_numpy() / 1200

for tar in weights:
    submission[tar] = submission[tar] * weights[tar]

submission.head()

submission.to_csv('submission.csv', index=False)

19532/19532 ━━━━━━━━━━━━━━━━━━━━ 35s 2ms/step
